<a href="https://colab.research.google.com/github/leul12/Data_Exploration/blob/main/tweet_stalking_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import itertools
from datetime import datetime
import datetime
import time
from pandas.io.json import json_normalize
import json
from tqdm.notebook import tqdm_notebook
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="darkgrid")

## Let's Check the version 🐍

In [3]:
from platform import python_version
print(python_version())

3.8.16


## Okay now we can do our **job**

1.   Let's install the 🔨
2.   Extract the 🗄️
3.   And So on



# ⬆️ install the tool 🔨

In [4]:
pip install git+https://github.com/JustAnotherArchivist/snscrape.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/JustAnotherArchivist/snscrape.git to /tmp/pip-req-build-_4sa8yxv
  Running command git clone -q https://github.com/JustAnotherArchivist/snscrape.git /tmp/pip-req-build-_4sa8yxv
  Created wheel for snscrape: filename=snscrape-0.4.3.20220107.dev73+gcbdaee6-py3-none-any.whl size=68025 sha256=fd42e63aa1cebf9f412cb151ce695dce76b7ae889d8d618d4c3a02e15433b69a
  Stored in directory: /tmp/pip-ephem-wheel-cache-oc1d3i5_/wheels/92/42/87/33fa9b18f7a75d02643a9ca3743339aec9be28c6796267c7d8
Successfully built snscrape


## 🤔   **how to extract the tweets ?** no worries ⬇️

In [5]:
import snscrape.modules.twitter as sntwitter

# okay now some magic to get the actual 🐦 using the **itertools** 😉

In [19]:
df = pd.DataFrame(itertools.islice(
    sntwitter.TwitterSearchScraper('#vaccine').get_items(), 30))
# let's Test what kind of data we can get from it (if it works)
df.head(2)

,url,date,rawContent,renderedContent,id,user,replyCount,retweetCount,likeCount,quoteCount,...,retweetedTweet,quotedTweet,inReplyToTweetId,inReplyToUser,mentionedUsers,coordinates,place,hashtags,cashtags,card
0,https://twitter.com/n7zzt/status/1610240331328...,2023-01-03 11:43:06+00:00,@MalliardReport about the only “mass extinctio...,@MalliardReport about the only “mass extinctio...,1610240331328655360,"{'username': 'n7zzt', 'id': 229675967, 'displa...",0,0,0,0,...,None,None,1.610224e+18,"{'username': 'MalliardReport', 'id': 451136428...","[{'username': 'MalliardReport', 'id': 45113642...",None,None,[Vaccine],None,None
1,https://twitter.com/BenAltstein/status/1610239...,2023-01-03 11:40:28+00:00,#COVIDー19 #COVID19 #COVIDIsAirborne #Vaccines ...,#COVIDー19 #COVID19 #COVIDIsAirborne #Vaccines ...,1610239668842283009,"{'username': 'BenAltstein', 'id': 955550968434...",0,1,2,0,...,None,{'url': 'https://twitter.com/DrEliDavid/status...,NaN,None,None,None,None,"[COVIDー19, COVID19, COVIDIsAirborne, Vaccines,...",None,None


# okay Now that we got this out of the way what now 🤔


*   we check data heads
*   clean some columns [user , hastags , date , mentionedUsers & inReplyToUser ]
*   and take it form there 
*   🚶‍♂️



In [25]:
dfuser= pd.json_normalize(df['user'])
dfuser.head(2)

,username,id,displayname,rawDescription,renderedDescription,descriptionLinks,verified,created,followersCount,friendsCount,...,location,protected,profileImageUrl,profileBannerUrl,label,link.text,link.url,link.tcourl,link.indices,link
0,n7zzt,229675967,Eric Oyen,#Constitutionalist #EagleScout #BlindAthlete #...,#Constitutionalist #EagleScout #BlindAthlete #...,"[{'text': 'twolifepaths.com', 'url': 'http://w...",False,2010-12-23 01:27:46+00:00,6356,6663,...,Phoenix Arizona,False,https://pbs.twimg.com/profile_images/104621646...,None,None,facebook.com/BlindAthletesC…,https://www.facebook.com/BlindAthletesCorner,https://t.co/epWuEGWiKd,"(0, 23)",NaN
1,BenAltstein,955550968434905090,Nudge Nudge News,Expose fuckery with facts. Ideas parsed from c...,Expose fuckery with facts. Ideas parsed from c...,None,False,2018-01-22 21:21:18+00:00,2734,1833,...,A misological bubble,False,https://pbs.twimg.com/profile_images/159825836...,https://pbs.twimg.com/profile_banners/95555096...,None,fuckery.com,http://fuckery.com,https://t.co/M4KGkVqI6K,"(0, 23)",NaN


# NOw Let's Take what we need and add the to the main Data Frame ⚙️

In [26]:
dfuser.drop(['id','displayname','rawDescription','renderedDescription','descriptionLinks','location','statusesCount','favouritesCount','listedCount',
                  'mediaCount','link','profileImageUrl','profileBannerUrl','label','link.text','link.url','link.tcourl','link.indices'], axis = 1, inplace = True)
dfuser.head(2)

,username,verified,created,followersCount,friendsCount,protected
0,n7zzt,False,2010-12-23 01:27:46+00:00,6356,6663,False
1,BenAltstein,False,2018-01-22 21:21:18+00:00,2734,1833,False


# now that we 🫧 the 🧫  time to add it to the clean data sets

In [27]:
df2=df.drop(['user','renderedContent','sourceUrl','hashtags','cashtags','card','inReplyToTweetId','inReplyToUser','mentionedUsers','coordinates','links','media','retweetedTweet','quotedTweet','place'],axis=1)
df2.head(2)

,url,date,rawContent,id,replyCount,retweetCount,likeCount,quoteCount,conversationId,lang,source,sourceLabel
0,https://twitter.com/n7zzt/status/1610240331328...,2023-01-03 11:43:06+00:00,@MalliardReport about the only “mass extinctio...,1610240331328655360,0,0,0,0,1610223668499550209,en,"<a href=""http://sites.google.com/site/yorufuku...",YoruFukurou
1,https://twitter.com/BenAltstein/status/1610239...,2023-01-03 11:40:28+00:00,#COVIDー19 #COVID19 #COVIDIsAirborne #Vaccines ...,1610239668842283009,0,1,2,0,1610239668842283009,qht,"<a href=""http://twitter.com/download/iphone"" r...",Twitter for iPhone


In [28]:
result = pd.concat([df2, dfuser], axis=1)
result.head(2)

,url,date,rawContent,id,replyCount,retweetCount,likeCount,quoteCount,conversationId,lang,source,sourceLabel,username,verified,created,followersCount,friendsCount,protected
0,https://twitter.com/n7zzt/status/1610240331328...,2023-01-03 11:43:06+00:00,@MalliardReport about the only “mass extinctio...,1610240331328655360,0,0,0,0,1610223668499550209,en,"<a href=""http://sites.google.com/site/yorufuku...",YoruFukurou,n7zzt,False,2010-12-23 01:27:46+00:00,6356,6663,False
1,https://twitter.com/BenAltstein/status/1610239...,2023-01-03 11:40:28+00:00,#COVIDー19 #COVID19 #COVIDIsAirborne #Vaccines ...,1610239668842283009,0,1,2,0,1610239668842283009,qht,"<a href=""http://twitter.com/download/iphone"" r...",Twitter for iPhone,BenAltstein,False,2018-01-22 21:21:18+00:00,2734,1833,False


# ⏱️ manuplation

In [32]:
result.set_index('date').head(2)

,url,rawContent,id,replyCount,retweetCount,likeCount,quoteCount,conversationId,lang,source,sourceLabel,username,verified,created,followersCount,friendsCount,protected
date,,,,,,,,,,,,,,,,,
2023-01-03 11:43:06+00:00,https://twitter.com/n7zzt/status/1610240331328...,@MalliardReport about the only “mass extinctio...,1610240331328655360,0,0,0,0,1610223668499550209,en,"<a href=""http://sites.google.com/site/yorufuku...",YoruFukurou,n7zzt,False,2010-12-23 01:27:46+00:00,6356,6663,False
2023-01-03 11:40:28+00:00,https://twitter.com/BenAltstein/status/1610239...,#COVIDー19 #COVID19 #COVIDIsAirborne #Vaccines ...,1610239668842283009,0,1,2,0,1610239668842283009,qht,"<a href=""http://twitter.com/download/iphone"" r...",Twitter for iPhone,BenAltstein,False,2018-01-22 21:21:18+00:00,2734,1833,False
